# Zero

Today we are going to start an experiment. LLMs have supplanted the need for stats NLP, Gemini is busted, and Sama is raising $7T. The president is eating ice cream and talking about a ceasefire. We're two years into this AI thing and I need to make some money. In this notebook. With you, dear reader, and the LLMs, and all the bots, past present and future, Copilot and GPT-4, Mistral & llama too. I hope you can run this locally, it's hard to say today where the compute will fall. I'm currently writing in Cursor, which means my AI du jour is some GPT flavor. The date is February 27, 2024.

I want to think of this as a group exercise. It's me, Peter, at the keys, but every time I hit tab, *boom* one of my accompanists will take the wheel. I haven't done it yet, and I'll try my best to use it sparingly, but we gotta make some money, and _I'm not sure how else to do it._

Goodnight.

In [3]:
!git init
!git add chapter-00.ipynb
!git commit -m "init"

Initialized empty Git repository in /Users/p/Desktop/zero/.git/
[main (root-commit) 00aad59] init
 1 file changed, 60 insertions(+)
 create mode 100644 chapter-00.ipynb


First instinct to make money with LLMs is: scrape FT, drudge, scan for company names, get their performance in that day, fine tune _a model to predict the performance of a company based on the news, and then trade on that._

Or we could do macro-vibe trading, take the above the fold headlines, line them up with S&P opens and closes, finetune, predict. Companies are much less complex units though, and if they're on a front page the signal is strong. Let's stick with that.

Okay, so we need a website snapshotter that'll get past bot detections. But we also need training data, so we'll use the wayback machine. Time for some python.

In [8]:
!pip install waybackpy beautifulsoup4

2007.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.7 MB/s eta 0:00:00
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
# use waybackpy to get the last year of drudge, every day at 9 AM ET. 
from waybackpy import WaybackMachineCDXServerAPI

url = "https://drudgereport.com/"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
drudge_wayback = WaybackMachineCDXServerAPI(url, user_agent)

newest_drudge_url = drudge_wayback.newest().archive_url

import requests
from bs4 import BeautifulSoup

# Fetch the page content
response = requests.get(newest_drudge_url)

# Parse the page content
soup = BeautifulSoup(response.text, 'html.parser')
# Find all 'a' tags (which define hyperlinks)
links = soup.find_all('a')

# Extract the actual link with 'href' and the text content
article_links = [(link.get('href'), link.text) for link in links]
# remove ones with blank text
article_links = [link for link in article_links if link[1]]

for url, text in article_links[:5]:
    print(f"URL: {url}, Text: {text}")

URL: https://web.archive.org/web/20240216111700/https://www.mirror.co.uk/news/us-news/breaking-kansas-city-chiefs-parade-32135595  , Text: Chiefs parade shooting 'personal dispute' and not terrorism as 2 'juveniles' arrested...
URL: https://web.archive.org/web/20240216111700/https://www.the-sun.com/sport/10384294/kansas-city-chiefs-parade-2024-live-super-bowl/, Text: 23 victims; Half are children...
URL: https://web.archive.org/web/20240216111700/https://metro.co.uk/2024/02/15/travis-kelce-heartbroken-kansas-city-chiefs-parade-shooting-20284137/, Text: Travis Kelce blasted for partying at bar hours after... 
URL: https://web.archive.org/web/20240216111700/https://apnews.com/article/fbdff18df40920b75873b3a40317f5ee, Text: FIRST CRIMINAL TRIAL SET FOR MARCH
URL: https://web.archive.org/web/20240216111700/https://www.cnn.com/2024/02/15/politics/trump-state-prosecutions-new-york-georgia/index.html, Text: LEGAL DRAMA IN TWO CITIES


In [18]:
# functionalize the above, passing the url, a year, month, day, hour, and minute, returning a list of tuples of url and text
def get_drudge_links(url, year, month, day, hour, minute):
    w = WaybackMachineCDXServerAPI(url, user_agent=user_agent)
    url = w.near(year=year, month=month, day=day, hour=hour, minute=minute).archive_url
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    article_links = [(link.get('href'), link.text) for link in links]
    article_links = [link for link in article_links if link[1]]
    return article_links

links = get_drudge_links("https://drudgereport.com/", year=2023, month=2, day=27, hour=9, minute=0)

print(links[:5])


[('https://web.archive.org/web/20230227080718/https://www.express.co.uk/news/world/1739632/russia-news-putin-andrey-sidelnikov-congress-people-deputies-ukraine-war-latest',
  'Russia braces for civil chaos...'),
 ('https://web.archive.org/web/20230227080718/https://www.mirror.co.uk/news/world-news/putin-will-die-office-bullets-29316932',
  "Putin 'will die in office or with bullets in his back'..."),
 ('https://web.archive.org/web/20230227080718/https://www.theguardian.com/world/2023/feb/25/thats-my-neighbour-mariupol-residents-shock-at-putins-parade-line-up',
  "Kremlin wheels out Ukrainian kids to 'thank' invaders..."),
 ('https://web.archive.org/web/20230227080718/https://www.barrons.com/news/us-poland-and-germany-may-hold-joint-manoeuvres-minister-f8c6edf',
  'US, Poland and Germany may hold joint maneuvers...'),
 ('https://web.archive.org/web/20230227080718/https://www.mirror.co.uk/news/world-news/inside-zelenskys-war-rooms-ukrainian-29316970',
  "Inside Zelensky's war rooms..."),